# Aggregate round/request results into tables

### 1) Overall configurations 

In [1]:
import os
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/instance_settings_test_rebalancing.json"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/allow_hiring.json"

##### Data translation

dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_sq_class = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)

# Vehicle type
dict_service = {"FLEET":"Company-owned", "FREELANCE":"Third-party"}
category_fleet = pd.api.types.CategoricalDtype(categories=["Company-owned", "Third-party"], ordered=True)

# Pickup deadline status (Was user chosen to have its pk deadline extended?)
dict_sl_status = {"MET":"Shortest", "UNMET":"Extended"}
category_status = pd.api.types.CategoricalDtype(categories=["Shortest", "Extended"], ordered=True)

# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {"3600":"1h", "0":"Single-ride", "10800":"3h", "-1":"-",  "BASELINE":"Baseline"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3", "BASELINE":"Baseline"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)


### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'instance_description': 'Whole week experiment (hiring and deteriorating '
                         'enabled)',
 'instance_name': 'HIRINGWEEKMAXWAITINGREB2',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'ID': 'instance_description',
            'IF': 'initial_fleet',
            'IN': 'instance_name',
            'MC': 'max_capacity',
            'MO': 'allow_many_to_one',
            'MR': 'max_requests',
            'RE': 'rebalance',
            'RT': 'reinsert_targets',
            'SD': 'allow_service_deterioration',
            'SR': 'service_rate',
            'ST': 'simulation_time',
            'UR': 'allow_urgent_relocation',
            'VH': 'allow_vehicle_hiring'},
 'rebala

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [3]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    # print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET}

In [4]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

### Processing all instances in folder (mean time)

In [5]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)


instance_attribute = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 2:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
     
    # Filter instance label
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    
    # Filter user base
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
    
    # Filter service rate
    if instance_settings.get('service_rate', None) in ['S1']:
        continue
        
    print("  - Processing", instance)
    
    # Load results
    experiment_file = "{}{}.csv".format(request_log_folder, instance)
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    column = ('', 'contract_duration')
    # When no contract duration is available, instance refers to baseline
    instance_attribute[column].append(int(instance_settings.get('contract_duration', '-1')))

    column = ('', 'customer_segmentation')
    instance_attribute[column].append(instance_settings['customer_segmentation'])

   
    # Convert denied to freelance to show denied under same header of the initial fleet
    df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
    
    
    for sq_class in ['A', 'B', 'C']:
        for sl_status in ["MET", "UNMET"]:
            # Categories leading to max, mean and count
            super_indexes = (dict_sq_class[sq_class], dict_sl_status[sl_status])

            # Filter dataframe
            filter_sq_sl = (df['class'] == sq_class) & (df['service_level'] == sl_status)
            df_filtered = df.loc[filter_sq_sl]["pk_delay"]
            
            mean_pk = df_filtered.mean()

            # Format data
            mean_pk_min = (mean_pk if not math.isnan(mean_pk) else 0)
            instance_attribute[super_indexes].append(mean_pk_min)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/request_track/'

In [ ]:
from datetime import datetime, timedelta
#pprint(instance_attribute)

# Build data frame from dictionary
a = pd.DataFrame.from_dict(instance_attribute)
a

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [ ]:

key_cs = ('', 'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('',  'contract_duration') #(0, 3600, 18000)

# Filtering data (only mixed segmentations scenarios will be used)
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[str(e)])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a = a.sort_values(by=[key_cs,key_cd])
a = a.set_index([key_cs,key_cd])

# Changing table column index names
a.index.names = ["User base", "Contract duration"]

# Transforming to minutes
#a = a.applymap(lambda e: ('{:.2f}'.format(float(e)/60) if e != '-' else e))
a = a.applymap(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))
a

### Getting latex table (median delay)

In [ ]:
print(a.to_latex(multicolumn=True, multirow=True))

### Processing all instances (count)

In [ ]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)

sl_status = "MET"

instance_attribute_count = defaultdict(list)
instance_attribute_count_percentage = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 0:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
        
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
    
    # Filter service rate
    if instance_settings.get('service_rate', None) in ['S1']:
        continue
        
    #if instance_settings['instance_name'] != "WEEKDENY" or 'rebalance' not in instance_settings.keys():
    #    continue
    
    print("  - Processing", instance)
    
    try:
        # Load results
        experiment_file = "{}{}.csv".format(request_log_folder, instance)
        df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
        column = ('', 'contract_duration')
        # When no contract duration is available, instance refers to baseline
        instance_attribute_count[column].append(instance_settings.get('contract_duration', 'BASELINE'))
        instance_attribute_count_percentage[column].append(instance_settings.get('contract_duration', 'BASELINE'))
            
        column = ('', 'customer_segmentation')
        instance_attribute_count[column].append(instance_settings['customer_segmentation'])
        instance_attribute_count_percentage[column].append(instance_settings['customer_segmentation'])
        
        # Get total number of requests
        total = len(df)
        
        # Convert denied to freelance to show denied under same header of the initial fleet
        df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
        
        print(df.service.value_counts())
        
        for sq_class in ['A', 'B', 'C']:
            for fleet in ['FLEET', 'FREELANCE']:

                # Categories leading to max, mean and count
                #super_indexes = (dict_service[fleet], dict_sq_class[sq_class], dict_sl_status[sl_status])
                #super_indexes = (dict_service[fleet], dict_sl_status[sl_status], dict_sq_class[sq_class])
                super_indexes = (dict_sq_class[sq_class], dict_service[fleet])

                # Filter dataframe
                filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                df_filtered = df.loc[filter_sq_sl]["pk_delay"]

                mean_pk = df_filtered.mean()
                max_pk = df_filtered.max()

                count = df_filtered.count()
                instance_attribute_count[super_indexes].append(count)

                count = (count/total if not (math.isnan(count) or count==0) else 0)
                instance_attribute_count_percentage[super_indexes].append(count)

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                # instance_attribute[(*super_indexes, 'Mean.')].append(mean_pk_min)

                print(instance_attribute_count_percentage)
        for fleet in ['FLEET', 'FREELANCE']:

            filter_sq_sl = (df['service'] == fleet) & (df['service_level'] == sl_status)
            subtotal_index = ("Total", dict_service[fleet])
            
            count_fleet = len(df.loc[filter_sq_sl])
            instance_attribute_count[subtotal_index].append(count_fleet)

            count_fleet = (count_fleet/total if not (math.isnan(count_fleet) or count_fleet==0) else 0)
            instance_attribute_count_percentage[subtotal_index].append(count_fleet)
            
    except Exception as e:
        print(e)
        pass

In [ ]:
instance_attribute_count_percentage

### Table: How many users were serviced by each vehicle type?

In [ ]:
# Build data frame from dictionary
b = pd.DataFrame.from_dict(instance_attribute_count_percentage)
key_cs = ('',  'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('',  'contract_duration') #(0, 3600, 18000)

# Renaming data and applying aliases
b[key_cs] = b[key_cs].map(lambda e:dict_segmentation[e])
b[key_cs] = b[key_cs].astype(category_segmentation)

b[key_cd] = b[key_cd].map(lambda e:dict_contract_duration[e])
b[key_cd] = b[key_cd].astype(category_contract_duration)

b = b.sort_values(by=[key_cs,key_cd])
b = b.set_index([key_cs,key_cd])
b.index.names = ["User base", "Contract duration"]


def formatResult(e):
    if e == 0:
        return '-'
    if e < 0.01:
        return '*'
    return '{:.2%}'.format(float(e))

overall_total = b.copy()
# Transforming to minutes
b = b.applymap(lambda e: formatResult(e))
b

In [ ]:
overall_total[("", "Total")]=overall_total[("Total", 'Company-owned')] + overall_total[("Total", 'Third-party')]
overall_total = overall_total.applymap(lambda e: formatResult(e))
overall_total

In [ ]:
print(overall_total.to_latex(multicolumn=True, multirow=True))

In [ ]:
b.to_csv("Counts.csv")

In [ ]:
file = "IN-WEEKDENY_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR.csv"

user = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True, comment="#")

In [ ]:
user.service_level.value_counts()

In [ ]:
if 'Company-ownedd' in ('Low-cost', 'Company-owned', 'Extended'):
    print("e")

In [ ]:
c = pd.DataFrame(b)

In [ ]:
extended[[(a, b, c) for a in ['Business', 'Standard', 'Low-cost'] for b in ['Company-owned', 'Third-party'] for c in ['Extended']]]